<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/xla/tutorials/autoclustering_xla"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/tensorflow/blob/master/tensorflow/compiler/xla/g3doc/tutorials/autoclustering_xla.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/tensorflow/blob/master/tensorflow/compiler/xla/g3doc/tutorials/autoclustering_xla.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

# Classifying CIFAR-10 with XLA

This tutorial trains a TensorFlow model to classify the [CIFAR-10](https://en.wikipedia.org/wiki/CIFAR-10) dataset, and we compile it using XLA.

Load and normalize the dataset using the Keras API:

In [1]:
import tensorflow as tf

# Check that GPU is available: cf. https://colab.research.google.com/notebooks/gpu.ipynb
assert(tf.test.gpu_device_name())

tf.keras.backend.clear_session()
tf.config.optimizer.set_jit(False) # Start with XLA disabled.

def load_data():
  (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
  x_train = x_train.astype('float32') / 256
  x_test = x_test.astype('float32') / 256

  # Convert class vectors to binary class matrices.
  y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
  y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)
  return ((x_train, y_train), (x_test, y_test))

(x_train, y_train), (x_test, y_test) = load_data()

170500096/170498071 [==============================] - 3s 0us/step


We define the model, adapted from the Keras [CIFAR-10 example](https://keras.io/examples/cifar10_cnn/):

In [2]:
def generate_model():
  return tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), padding='same', input_shape=x_train.shape[1:]),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Conv2D(32, (3, 3)),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(0.25),

    tf.keras.layers.Conv2D(64, (3, 3), padding='same'),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Conv2D(64, (3, 3)),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(0.25),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10),
    tf.keras.layers.Activation('softmax')
  ])

model = generate_model()

We train the model using the
[RMSprop](https://www.tensorflow.org/api_docs/python/tf/train/RMSPropOptimizer)
optimizer:


In [3]:
def compile_model(model):
  opt = tf.keras.optimizers.RMSprop(lr=0.0001, decay=1e-6)
  model.compile(loss='categorical_crossentropy',
                optimizer=opt,
                metrics=['accuracy'])
  return model

model = compile_model(model)

def train_model(model, x_train, y_train, x_test, y_test, epochs=25):
  model.fit(x_train, y_train, batch_size=256, epochs=epochs, validation_data=(x_test, y_test), shuffle=True)

def warmup(model, x_train, y_train, x_test, y_test):
  # Warm up the JIT, we do not wish to measure the compilation time.
  initial_weights = model.get_weights()
  train_model(model, x_train, y_train, x_test, y_test, epochs=1)
  model.set_weights(initial_weights)

warmup(model, x_train, y_train, x_test, y_test)
%time train_model(model, x_train, y_train, x_test, y_test)

scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

196/196 [==============================] - 37s 22ms/step - loss: 2.1783 - accuracy: 0.1819 - val_loss: 1.7990 - val_accuracy: 0.3697
Epoch 1/25
196/196 [==============================] - 4s 19ms/step - loss: 2.1248 - accuracy: 0.2129 - val_loss: 1.8769 - val_accuracy: 0.3488
Epoch 2/25
196/196 [==============================] - 4s 19ms/step - loss: 1.8139 - accuracy: 0.3480 - val_loss: 1.6891 - val_accuracy: 0.4037
Epoch 3/25
196/196 [==============================] - 4s 19ms/step - loss: 1.6824 - accuracy: 0.3933 - val_loss: 1.6299 - val_accuracy: 0.4216
Epoch 4/25
196/196 [==============================] - 4s 19ms/step - loss: 1.5943 - accuracy: 0.4261 - val_loss: 1.5702 - val_accuracy: 0.4414
Epoch 5/25
196/196 [==============================] - 4s 19ms/step - loss: 1.5300 - accuracy: 0.4491 - val_loss: 1.5382 - val_accuracy: 0.4522
Epoch 6/25
196/196 [==============================] - 4s 19ms/step - loss: 1.4774 - accuracy: 0.4692 - val_loss: 1.3922 - val_accuracy: 0.5061
Epoch 7/2

Now let's train the model again, using the XLA compiler.
To enable the compiler in the middle of the application, we need to reset the Keras session.

In [4]:
# We need to clear the session to enable JIT in the middle of the program.
tf.keras.backend.clear_session()
tf.config.optimizer.set_jit(True) # Enable XLA.
model = compile_model(generate_model())
(x_train, y_train), (x_test, y_test) = load_data()

warmup(model, x_train, y_train, x_test, y_test)
%time train_model(model, x_train, y_train, x_test, y_test)

196/196 [==============================] - 8s 24ms/step - loss: 2.1652 - accuracy: 0.1917 - val_loss: 1.9119 - val_accuracy: 0.3259
Epoch 1/25
196/196 [==============================] - 6s 29ms/step - loss: 2.1076 - accuracy: 0.2214 - val_loss: 1.8558 - val_accuracy: 0.3557
Epoch 2/25
196/196 [==============================] - 4s 19ms/step - loss: 1.7986 - accuracy: 0.3518 - val_loss: 1.7106 - val_accuracy: 0.3933
Epoch 3/25
196/196 [==============================] - 4s 19ms/step - loss: 1.6694 - accuracy: 0.3951 - val_loss: 1.6572 - val_accuracy: 0.4022
Epoch 4/25
196/196 [==============================] - 4s 19ms/step - loss: 1.5751 - accuracy: 0.4296 - val_loss: 1.4842 - val_accuracy: 0.4647
Epoch 5/25
196/196 [==============================] - 4s 19ms/step - loss: 1.5181 - accuracy: 0.4494 - val_loss: 1.5126 - val_accuracy: 0.4583
Epoch 6/25
196/196 [==============================] - 4s 19ms/step - loss: 1.4707 - accuracy: 0.4702 - val_loss: 1.4382 - val_accuracy: 0.4848
Epoch 7/25

On a machine with a Titan V GPU and an Intel Xeon E5-2690 CPU the speed up is ~1.17x.